In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\PADHAI MMA FALL SEM\MMA\Winter sem 2\Fatih ML 2\ML_data.csv')

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\sahil\AppData\Local\Temp\ipykernel_13884\4128556153.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_csv('C:\PADHAI MMA FALL SEM\MMA\Winter sem 2\Fatih ML 2\ML_data.csv')


In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06 UTC,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT
1,2020-09-24 11:57:26 UTC,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY
2,2020-09-24 11:57:27 UTC,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy
3,2020-09-24 11:57:33 UTC,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08
4,2020-09-24 11:57:36 UTC,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ


                           +-----------------+
                           | Raw Events Table |  (Bronze Layer)
                           +-----------------+
                                   ↓
                      Session Aggregation, Feature Engineering
                                   ↓
                           +-----------------+
                           | Aggregated Data  |  (Silver Layer)
                           +-----------------+
                                   ↓
         +---------------------+         +--------------------------+
         | Gold Table for       |         | Gold Table for            |
         | Multi-Class Action   |         | Customer Behavior         |
         +---------------------+         +--------------------------+
                 ↓                                    ↓
       Train Multi-Class Model                 Train Clustering Model
      (TabNet, XGBoost, CatBoost)              (KMeans, HDBSCAN Clustering)
                 ↓                                    ↓
       Train Meta-Classifier                     Train Classifier on Clusters
     (Stacked Ensemble)                          (optional supervised learning)
                 ↓                                    ↓
        MLflow Model Registry                    MLflow Model Registry


In [4]:
# Load and basic clean
df['event_time'] = pd.to_datetime(df['event_time'])
df['brand'].fillna('unknown', inplace=True)
df['category_code'].fillna('unknown', inplace=True)
df = df[df['price'] > 0]

# Sort events
df = df.sort_values(['user_id', 'event_time'])

C:\Users\sahil\AppData\Local\Temp\ipykernel_13884\848689186.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['brand'].fillna('unknown', inplace=True)
C:\Users\sahil\AppData\Local\Temp\ipykernel_13884\848689186.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [8]:
df['next_event'] = df.groupby('user_id')['event_type'].shift(-1)

# Label = Last event_type in session.
# If using custom sessions: create session_id, and for each session, pick the last event's event_type as label.

In [5]:
# Time difference from previous event (per user)
df['prev_event_time'] = df.groupby('user_id')['event_time'].shift(1)
df['time_diff_minutes'] = (df['event_time'] - df['prev_event_time']).dt.total_seconds() / 60

# New session if time_diff > 30 minutes or first event
df['new_session'] = (df['time_diff_minutes'] > 30) | (df['time_diff_minutes'].isna())

# Cumulative sum of new sessions to assign session_id
df['session_id'] = df.groupby('user_id')['new_session'].cumsum()

# Combine user_id + session_id to make global session_id
df['session_id'] = df['user_id'].astype(str) + '_' + df['session_id'].astype(int).astype(str)

# Drop helper columns
df.drop(['prev_event_time', 'time_diff_minutes', 'new_session'], axis=1, inplace=True)

print(df.head())

                      event_time event_type  product_id          category_id  \
174670 2020-10-29 11:28:35+00:00       view     4101974  2144415939364389423   
51155  2020-10-06 06:30:32+00:00       view     3506650  2144415935673401802   
242211 2020-11-09 08:52:51+00:00       view      124883  2144415924424278172   
242297 2020-11-09 09:04:34+00:00       view      760044  2144415924424278172   
242335 2020-11-09 09:08:53+00:00       view      125325  2144415924424278172   

                     category_code     brand  price              user_id  \
174670          electronics.clocks     honor  76.48  1515915625353226922   
51155                      unknown    kester  28.98  1515915625353230067   
242211  electronics.audio.acoustic  logitech  23.90  1515915625353230683   
242297  electronics.audio.acoustic      dell  48.73  1515915625353230683   
242335  electronics.audio.acoustic  logitech  23.90  1515915625353230683   

       user_session             session_id  
174670   7qejzWzH

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 885129 entries, 174670 to 885127
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   event_time     885129 non-null  datetime64[ns, UTC]
 1   event_type     885129 non-null  object             
 2   product_id     885129 non-null  int64              
 3   category_id    885129 non-null  int64              
 4   category_code  885129 non-null  object             
 5   brand          885129 non-null  object             
 6   price          885129 non-null  float64            
 7   user_id        885129 non-null  int64              
 8   user_session   884964 non-null  object             
 9   session_id     885129 non-null  object             
 10  next_event     477846 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(3), object(6)
memory usage: 81.0+ MB


##################################################################################################################

#
Feature Engineering
#

For each user:
If time difference between consecutive events > 30 minutes → new session

In [7]:
# Feature aggregation per session
agg_funcs = {
    'event_type': [
        ('total_views', lambda x: (x == 'view').sum()),
        ('total_carts', lambda x: (x == 'cart').sum()),
        ('total_purchases', lambda x: (x == 'purchase').sum())
    ],
    'price': [
        ('avg_price', 'mean'),
        ('price_std', 'std'),
    ],
    'event_time': [
        ('session_start', 'min'),
        ('session_end', 'max'),
        ('avg_hour', lambda x: x.dt.hour.mean())
    ],
    'category_code': [
        ('most_freq_category', lambda x: x.mode()[0] if not x.mode().empty else 'unknown')
    ],
    'brand': [
        ('most_freq_brand', lambda x: x.mode()[0] if not x.mode().empty else 'unknown')
    ]
}

# Reformatting for Pandas GroupBy
agg_dict = {}
for col, operations in agg_funcs.items():
    for name, func in operations:
        agg_dict[f"{col}__{name}"] = (col, func)

# Perform aggregation
df_agg = df.groupby('session_id').agg(**agg_dict).reset_index()

# Post-processing
df_agg['session_duration_secs'] = (df_agg['event_time__session_end'] - df_agg['event_time__session_start']).dt.total_seconds()
df_agg['events_per_minute'] = (df_agg['event_type__total_views'] + df_agg['event_type__total_carts'] + df_agg['event_type__total_purchases']) / (df_agg['session_duration_secs'] / 60 + 1)

# Ratios
df_agg['view_to_cart_ratio'] = df_agg['event_type__total_carts'] / (df_agg['event_type__total_views'] + 1)
df_agg['cart_to_purchase_ratio'] = df_agg['event_type__total_purchases'] / (df_agg['event_type__total_carts'] + 1)
df_agg['view_to_purchase_ratio'] = df_agg['event_type__total_purchases'] / (df_agg['event_type__total_views'] + 1)

# Fill missing std price
df_agg['price__price_std'].fillna(0, inplace=True)

# Rename cleanly
df_agg.columns = [col.replace('event_type__', '').replace('price__', '').replace('event_time__', '') for col in df_agg.columns]


C:\Users\sahil\AppData\Local\Temp\ipykernel_13884\3699480669.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_agg['price__price_std'].fillna(0, inplace=True)


Now we have 1 row per session_id with engineered features.

 Label Creation for Classifier |
 
Logic:

Find the last event in each session.

That event_type will be the label: view / cart / purchase.

In [10]:
# Get last event per session
df_last_event = df.sort_values(['user_id', 'session_id', 'event_time']).groupby('session_id').tail(1)
session_labels = df_last_event[['session_id', 'event_type']].rename(columns={'event_type': 'final_action'})
# Merge label with features
df_final = df_agg.merge(session_labels, on='session_id', how='left')
print(df_final.head())

              session_id  total_views  total_carts  total_purchases  \
0  1515915625353226922_1            1            0                0   
1  1515915625353230067_1            1            0                0   
2  1515915625353230683_1            8            0                0   
3  1515915625353230683_2            4            0                0   
4  1515915625353230683_3            1            0                0   

   avg_price  price_std             session_start               session_end  \
0    76.4800   0.000000 2020-10-29 11:28:35+00:00 2020-10-29 11:28:35+00:00   
1    28.9800   0.000000 2020-10-06 06:30:32+00:00 2020-10-06 06:30:32+00:00   
2    22.2725  12.484057 2020-11-09 08:52:51+00:00 2020-11-09 09:26:38+00:00   
3    74.2400   0.000000 2020-11-18 10:51:35+00:00 2020-11-18 10:59:26+00:00   
4   339.7900   0.000000 2020-12-12 10:33:09+00:00 2020-12-12 10:33:09+00:00   

   avg_hour category_code__most_freq_category brand__most_freq_brand  \
0    11.000               

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504679 entries, 0 to 504678
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype              
---  ------                             --------------   -----              
 0   session_id                         504679 non-null  object             
 1   total_views                        504679 non-null  int64              
 2   total_carts                        504679 non-null  int64              
 3   total_purchases                    504679 non-null  int64              
 4   avg_price                          504679 non-null  float64            
 5   price_std                          504679 non-null  float64            
 6   session_start                      504679 non-null  datetime64[ns, UTC]
 7   session_end                        504679 non-null  datetime64[ns, UTC]
 8   avg_hour                           504679 non-null  float64            
 9   category_code__most_freq_category  50

Gold TABLE 2 for the Clustering and customer segmentation.

In [11]:
# --- Start by reusing previous session-level aggregation

# df_agg was already created in previous step (session-level features)
# Now, drop final_action related stuff

gold_table_2 = df_agg.copy()

# Optional: add user_id back to session table
session_user_map = df[['session_id', 'user_id']].drop_duplicates()
gold_table_2 = gold_table_2.merge(session_user_map, on='session_id', how='left')

# Rearranging columns nicely
cols = ['user_id', 'session_id'] + [c for c in gold_table_2.columns if c not in ['user_id', 'session_id']]
gold_table_2 = gold_table_2[cols]

# Drop final_action because segmentation is unsupervised
if 'final_action' in gold_table_2.columns:
    gold_table_2.drop('final_action', axis=1, inplace=True)

print(gold_table_2.head())

               user_id             session_id  total_views  total_carts  \
0  1515915625353226922  1515915625353226922_1            1            0   
1  1515915625353230067  1515915625353230067_1            1            0   
2  1515915625353230683  1515915625353230683_1            8            0   
3  1515915625353230683  1515915625353230683_2            4            0   
4  1515915625353230683  1515915625353230683_3            1            0   

   total_purchases  avg_price  price_std             session_start  \
0                0    76.4800   0.000000 2020-10-29 11:28:35+00:00   
1                0    28.9800   0.000000 2020-10-06 06:30:32+00:00   
2                0    22.2725  12.484057 2020-11-09 08:52:51+00:00   
3                0    74.2400   0.000000 2020-11-18 10:51:35+00:00   
4                0   339.7900   0.000000 2020-12-12 10:33:09+00:00   

                session_end  avg_hour category_code__most_freq_category  \
0 2020-10-29 11:28:35+00:00    11.000                

In [12]:
# Aggregate sessions to user-level
gold_table_2_user = gold_table_2.groupby('user_id').agg({
    'total_views': 'sum',
    'total_carts': 'sum',
    'total_purchases': 'sum',
    'avg_price': 'mean',
    'price_std': 'mean',
    'session_duration_secs': 'mean',
    'events_per_minute': 'mean',
    'view_to_cart_ratio': 'mean',
    'cart_to_purchase_ratio': 'mean',
    'view_to_purchase_ratio': 'mean',
}).reset_index()

print(gold_table_2_user.head())

               user_id  total_views  total_carts  total_purchases   avg_price  \
0  1515915625353226922            1            0                0   76.480000   
1  1515915625353230067            1            0                0   28.980000   
2  1515915625353230683           13            0                0  145.434167   
3  1515915625353230922            1            0                0  274.400000   
4  1515915625353234047           36            0                0  148.662556   

   price_std  session_duration_secs  events_per_minute  view_to_cart_ratio  \
0   0.000000               0.000000           1.000000                 0.0   
1   0.000000               0.000000           1.000000                 0.0   
2   4.161352             832.666667           0.560658                 0.0   
3   0.000000               0.000000           1.000000                 0.0   
4  10.638771             158.200000           1.257183                 0.0   

   cart_to_purchase_ratio  view_to_purchase_

Multi Class Classification --> Trying to predict customers final action

In [13]:
from sklearn.model_selection import train_test_split

# Prepare data
X = df_final.drop(['session_id', 'final_action'], axis=1)
y = df_final['final_action']

# Train-Test split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504679 entries, 0 to 504678
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype              
---  ------                             --------------   -----              
 0   total_views                        504679 non-null  int64              
 1   total_carts                        504679 non-null  int64              
 2   total_purchases                    504679 non-null  int64              
 3   avg_price                          504679 non-null  float64            
 4   price_std                          504679 non-null  float64            
 5   session_start                      504679 non-null  datetime64[ns, UTC]
 6   session_end                        504679 non-null  datetime64[ns, UTC]
 7   avg_hour                           504679 non-null  float64            
 8   category_code__most_freq_category  504679 non-null  object             
 9   brand__most_freq_brand             50

Building Preprocessing Pipelines
1.->Scale numerical features
2.-> One-hot encode categorical features

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Identify columns
num_features = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_features = ['category_code__most_freq_category', 'brand__most_freq_brand']  # Categorical from aggregation

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

Model Definitions

TabNetClassifier | CatBoostClassifier | XGBoostClassifier

In [24]:
from pytorch_tabnet.tab_model import TabNetClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Base models
tabnet_model = TabNetClassifier(verbose=0)
catboost_model = CatBoostClassifier(verbose=0)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

Preprocessor + Model = Full pipeline

In [25]:
from sklearn.pipeline import make_pipeline

# Pipelines
tabnet_pipeline = make_pipeline(preprocessor, tabnet_model)
catboost_pipeline = make_pipeline(preprocessor, catboost_model)
xgb_pipeline = make_pipeline(preprocessor, xgb_model)

Hyperparameter Optimization using OPTUNA

We'll use Optuna to optimize each model separately

Include L1 and L2 penalties

L1 regularization for feature selection etc.

In [26]:
import torch

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [27]:
print(X_train.columns)

Index(['total_views', 'total_carts', 'total_purchases', 'avg_price',
       'price_std', 'session_start', 'session_end', 'avg_hour',
       'category_code__most_freq_category', 'brand__most_freq_brand',
       'session_duration_secs', 'events_per_minute', 'view_to_cart_ratio',
       'cart_to_purchase_ratio', 'view_to_purchase_ratio'],
      dtype='object')


In [29]:
import time
import optuna
import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [30]:
mlflow.set_experiment("TabNet_CatBoost_XGB_Tuning_GPU")

2025/04/26 21:30:08 INFO mlflow.tracking.fluent: Experiment with name 'TabNet_CatBoost_XGB_Tuning_GPU' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/PADHAI%20MMA%20FALL%20SEM/MMA/Winter%20sem%202/Fatih%20ML%202/mlruns/427491984796624465', creation_time=1745717408887, experiment_id='427491984796624465', last_update_time=1745717408887, lifecycle_stage='active', name='TabNet_CatBoost_XGB_Tuning_GPU', tags={}>

# Tabnet classifier with hyperparameter tuning using Optuna

In [43]:
import time
import mlflow
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def objective_tabnet(trial):
    start_time = time.time()
    params = {
        "n_d": trial.suggest_int("n_d", 16, 32),
        "n_a": trial.suggest_int("n_a", 16, 32),
        "n_steps": trial.suggest_int("n_steps", 3, 6),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-5, 1e-2, log=True), 
        "momentum": trial.suggest_float("momentum", 0.01, 0.4),
        "optimizer_params": {"lr": trial.suggest_float("lr", 1e-4, 1e-2, log=True)}, 
        "verbose": 0, 
        "device_name": 'cuda' if torch.cuda.is_available() else 'cpu' 
    }

    model = TabNetClassifier(**params)
    pipeline = make_pipeline(preprocessor, model)

    with mlflow.start_run(nested=True):
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)

        # Calculate evaluation metrics
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred, average='macro', zero_division=0) 
        rec = recall_score(y_val, y_pred, average='macro', zero_division=0)   
        f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)       

        duration_sec = time.time() - start_time
        mlflow.log_params(params)
        mlflow.log_metrics({
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1_score": f1,
            "duration_sec": duration_sec
        })
    print(
        f"Trial {trial.number} finished in {duration_sec:.2f} sec | "
        f"Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
        f"Recall: {rec:.4f}, F1 Score: {f1:.4f}"
    )
    return f1 

In [44]:
import time
import mlflow
from catboost import CatBoostClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch 

def objective_catboost(trial):
    start_time = time.time()

    # Suggest hyperparameters using Optuna
    params = {
        "iterations": 200,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 5),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 1),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        "random_strength": trial.suggest_float("random_strength", 1e-3, 1),
        "eval_metric": "MultiClass",
        "loss_function": "MultiClass",
        "task_type": "GPU" if torch.cuda.is_available() else "CPU", 
        "verbose": 0 
    }

    # Create and train the CatBoost model within a pipeline
    model = CatBoostClassifier(**params)
    # Assuming 'preprocessor' is a defined sklearn ColumnTransformer or similar
    pipeline = make_pipeline(preprocessor, model)

    with mlflow.start_run(nested=True):
        # Fit the pipeline
        pipeline.fit(X_train, y_train)

        # Predict on validation set
        y_pred = pipeline.predict(X_val)

        # Calculate evaluation metrics
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred, average='macro', zero_division=0) 
        rec = recall_score(y_val, y_pred, average='macro', zero_division=0)   
        f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)       

        duration_sec = time.time() - start_time

        # Log parameters and metrics to MLflow
        mlflow.log_params(params)
        mlflow.log_metrics({
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1_score": f1,
            "duration_sec": duration_sec
        })

    # Print metrics and time taken after the trial completes
    print(
        f"Trial {trial.number} finished in {duration_sec:.2f} sec | "
        f"Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
        f"Recall: {rec:.4f}, F1 Score: {f1:.4f}"
    )

    # Return the metric to be minimized/maximized by Optuna
    return f1

In [45]:
import time
import mlflow
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def objective_xgb(trial):

    start_time = time.time()

    params = {
        "n_estimators": 200, 
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
        "tree_method": "gpu_hist", 
        "use_label_encoder": False, 
        "eval_metric": "mlogloss",
        "random_state": 42 
    }

    model = XGBClassifier(**params)
    pipeline = make_pipeline(preprocessor, model)

    with mlflow.start_run(nested=True):
        # Fit the pipeline
        # Use early_stopping_rounds if you have a separate evaluation set for early stopping
        pipeline.fit(X_train, y_train, xgbclassifier__eval_set=[(X_val, y_val)], xgbclassifier__early_stopping_rounds=50)
        pipeline.fit(X_train, y_train)


        # Predict on validation set
        y_pred = pipeline.predict(X_val)

        # Calculate evaluation metrics
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred, average='macro', zero_division=0) 
        rec = recall_score(y_val, y_pred, average='macro', zero_division=0)   
        f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)       

        duration_sec = time.time() - start_time

        # Log parameters and metrics to MLflow
        mlflow.log_params(params)
        mlflow.log_metrics({
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1_score": f1,
            "duration_sec": duration_sec
        })

    print(
        f"Trial {trial.number} finished in {duration_sec:.2f} sec | "
        f"Metrics - Accuracy: {acc:.4f}, Precision: {prec:.4f}, "
        f"Recall: {rec:.4f}, F1 Score: {f1:.4f}"
    )
    return f1 

In [34]:
# Optuna study settings
n_trials = 30

# ========== Run TabNet ==========
print(" Tuning TabNet ")

study_tabnet = optuna.create_study(direction="maximize", study_name="TabNet_Tuning")
study_tabnet.optimize(objective_tabnet, n_trials=n_trials)

print(f" Best TabNet Score: {study_tabnet.best_value:.4f}")
print(f" Best TabNet Params: {study_tabnet.best_params}")

# ========== Run CatBoost ==========
print(" Tuning CatBoost ")

study_catboost = optuna.create_study(direction="maximize", study_name="CatBoost_Tuning")
study_catboost.optimize(objective_catboost, n_trials=n_trials)

print(f" Best CatBoost Score: {study_catboost.best_value:.4f}")
print(f" Best CatBoost Params: {study_catboost.best_params}")

# ========== Run XGBoost ==========
print(" Tuning XGBoost ")

study_xgb = optuna.create_study(direction="maximize", study_name="XGB_Tuning")
study_xgb.optimize(objective_xgb, n_trials=n_trials)

print(f" Best XGBoost Score: {study_xgb.best_value:.4f}")
print(f" Best XGBoost Params: {study_xgb.best_params}")


[I 2025-04-26 22:29:02,253] A new study created in memory with name: TabNet_Tuning


 Tuning TabNet 


C:\Users\sahil\AppData\Roaming\Python\Python313\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


In [ ]:
import json

best_params = {
    "tabnet": study_tabnet.best_params,
    "catboost": study_catboost.best_params,
    "xgboost": study_xgb.best_params,
}

with open("best_model_params.json", "w") as f:
    json.dump(best_params, f, indent=4)

print(" Best hyperparameters saved.")

In [ ]:
import optuna.visualization as vis

# Optimization history plots
vis.plot_optimization_history(study_tabnet).show()
vis.plot_optimization_history(study_catboost).show()
vis.plot_optimization_history(study_xgb).show()

# Parameter importance plots
vis.plot_param_importances(study_tabnet).show()
vis.plot_param_importances(study_catboost).show()
vis.plot_param_importances(study_xgb).show()

# Stacking Models.

### Build Final Individual Models (TabNet, CatBoost, XGBoost)

In [ ]:
import xgboost as xgb
# TabNet
tabnet_model = TabNetClassifier(**best_params['tabnet'])

# CatBoost
catboost_model = CatBoostClassifier(
    **best_params['catboost'], 
    task_type='GPU', devices='0'
)

# XGBoost
xgb_model = xgb.XGBClassifier(
    **best_params['xgboost'], 
    tree_method='gpu_hist', predictor='gpu_predictor', gpu_id=0
)

## StackingClassifier Ensemble

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

stacked_model = StackingClassifier(
    estimators=[
        ('tabnet', tabnet_model),
        ('catboost', catboost_model),
        ('xgb', xgb_model)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    stack_method='predict_proba',  
    n_jobs=-1,
    passthrough=True
)

stack_method='predict_proba' helps stacking better for multi-class.


passthrough=True sends original features + predictions to final estimator.

### Objective Function for Stacked Model

In [ ]:
def objective_stacking(trial):
    # Suggest C (regularization) for final Logistic Regression
    c = trial.suggest_loguniform('final_estimator_C', 1e-5, 10)

    stacked_model.final_estimator = LogisticRegression(C=c, max_iter=1000)

    with mlflow.start_run(nested=True):
        start_time = time.time()
        
        # Fit
        stacked_model.fit(X_train, y_train)

        preds = stacked_model.predict(X_val)

        acc = accuracy_score(y_val, preds)
        precision = precision_score(y_val, preds, average='weighted')
        recall = recall_score(y_val, preds, average='weighted')
        f1 = f1_score(y_val, preds, average='weighted')

        duration = time.time() - start_time

        # Log metrics and params
        mlflow.log_params({
            "stack_C": c,
        })
        mlflow.log_metrics({
            "accuracy": acc,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "duration": duration
        })

        return f1

#### Optuna for Stacking 

In [ ]:
study_stacking = optuna.create_study(direction="maximize", study_name="StackedModel_Tuning")
study_stacking.optimize(objective_stacking, n_trials=20)

print(f" Best Stacking Score: {study_stacking.best_value:.4f}")
print(f" Best Stacking Params: {study_stacking.best_params}")

#### Best Final Stacked Model to MLflow Model Registry

In [ ]:
# Re-train on full training set (train + valid)
best_c = study_stacking.best_params['final_estimator_C']
stacked_model.final_estimator = LogisticRegression(C=best_c, max_iter=1000)
stacked_model.fit(X_train, y_train)  # retrain on more data

# Log model
with mlflow.start_run(run_name="Final_Stacked_Model"):
    mlflow.sklearn.log_model(stacked_model, "stacked_model", registered_model_name="CustomerAction_StackedClassifier")
    print(" Final Stacked Model saved to MLflow Registry!")

##  Visualization

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

preds = stacked_model.predict(X_val)

ConfusionMatrixDisplay.from_predictions(y_val, preds)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import seaborn as sns

report = classification_report(y_val, preds, output_dict=True)
sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True)
plt.show()

In [ ]:
import optuna.visualization as vis

vis.plot_optimization_history(study_stacking).show()
vis.plot_param_importances(study_stacking).show()

# ##############################################################################################################

##################################################################################################################

# Customer Segmentation (clustering)

In [ ]:
gold_table_2_user.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

X = gold_table_2_user.drop(columns=["user_id"])

# Find Optimal Clusters Using Elbow or Silhouette
sil_scores = []
k_list = list(range(2, 11))

for k in k_list:
    model = KMeans(n_clusters=k, random_state=42)
    labels = model.fit_predict(X)
    sil = silhouette_score(X, labels)
    sil_scores.append(sil)

# Plot
import matplotlib.pyplot as plt

plt.plot(k_list, sil_scores, marker='o')
plt.xlabel("k clusters")
plt.ylabel("Silhouette Score")
plt.title("Finding Best k")
plt.show()

##### Pick the k with highest silhouette score

In [ ]:
kmeans_final = KMeans(n_clusters=4, random_state=42)
user_clusters = kmeans_final.fit_predict(X)

gold_table_2_user["cluster_label"] = user_clusters

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=30)
user_clusters_hdbscan = clusterer.fit_predict(X)

gold_table_2_user["cluster_label_hdbscan"] = user_clusters_hdbscan

In [ ]:
gold_table_2_user.groupby('cluster_label').mean()

#### 

Cluster 0 → "Bargain Hunter"

Cluster 1 → "Browser"

Cluster 2 → "Big Spender"

Cluster 3 → "Window Shopper"

In [ ]:
cluster_mapping = {
    0: "Bargain Hunter",
    1: "Browser",
    2: "Big Spender",
    3: "Window Shopper"
}

gold_table_2_user["customer_segment"] = gold_table_2_user["cluster_label"].map(cluster_mapping)

### Build a Classifier to Predict Segment

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Features and target
X_seg = gold_table_2_user.drop(columns=["user_id", "cluster_label", "customer_segment"])
y_seg = gold_table_2_user["customer_segment"]

# Train/Val Split
X_train_seg, X_val_seg, y_train_seg, y_val_seg = train_test_split(X_seg, y_seg, test_size=0.2, random_state=42)

# Model
rf_seg_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_seg_model.fit(X_train_seg, y_train_seg)

# Evaluate
preds_seg = rf_seg_model.predict(X_val_seg)
print(classification_report(y_val_seg, preds_seg))

In [ ]:
silhouette = silhouette_score(X, user_clusters)
print(f"Silhouette Score: {silhouette:.4f}")

In [ ]:
with mlflow.start_run(run_name="CustomerSegmentation"):
    mlflow.log_params({
        "Clustering Method": "KMeans",
        "n_clusters": 4,
    })
    mlflow.log_metrics({
        "silhouette_score": silhouette
    })
    
    mlflow.sklearn.log_model(rf_seg_model, "SegmentClassifier")

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

import seaborn as sns

plt.figure(figsize=(10,6))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=gold_table_2_user['customer_segment'], palette="Set2")
plt.title("User Segments via PCA")
plt.show()